# Project 1 - Orbit analysis with poliastro

![poliastro logo](https://docs.poliastro.space/en/latest/_images/logo_text.png)

**poliastro** is a Python library for interactive Astrodynamics. It provides a simple and intuitive API, handles physical quantities with units, and offers both 2D and 3D plotting. It is available under a permissive, commercial-friendly open source license (MIT).

Some of its capabilities are:

- Analytical and numerical orbit propagation
- Conversion between position and velocity vectors and classical orbital elements
- Hohmann and bielliptic maneuvers computation
- Natural perturbations
- Several optimal low-thrust guidance laws
- Trajectory plotting, porkchop plots, Tisserand graphs, groundtrack plots
- Initial orbit determination (Lambert problem)
- Easy loading of ephemerides for Solar System bodies

It leverages several powerful libraries, like numba, Astropy, SciPy, Plotly and others.

- **Documentation**: https://docs.poliastro.space/en/stable/
- **Source code**: https://github.com/poliastro/poliastro/

## `Orbit` and `Quantity` objects

The core of poliastro are the `Orbit` objects inside the `poliastro.twobody` module. They store all the required information to define an orbit:

- The body acting as the central body of the orbit, for example the Earth.
- The position and velocity vectors or the orbital elements.
- The time (epoch) at which the orbit is defined.

Let's load a example orbit of the International Space Station:

In [ ]:
from poliastro.examples import iss

If we display the object itself, it shows some basic information:

In [ ]:
iss

We can access individual orbital elements from this `Orbit`, for example:

In [ ]:
iss.a  # Semimajor axis

In [ ]:
iss.ecc  # Eccentricity

In [ ]:
iss.v  # Velocity vector

Notice that all these values have physical units, because they are `astropy.units.Quantity` objects:

In [ ]:
from astropy import units as u

In [ ]:
iss.r_p.to(u.m)  # Radius of periapsis, in meters

In [ ]:
iss.raan.to(u.deg)  # Right ascension of the ascending node, in degrees

And if we mix `Quantity`es with different units, we get the expected result:

In [ ]:
iss.attractor.R  # Radius of the attractor, in this case the Earth, in meters

In [ ]:
# 400 * u.km creates a Quantity object
iss.attractor.R + 400 * u.km  # Units are operated correctly

Besides, we can easily produce a schematic plot of the orbit. There are three kinds:

- Static, 2D
- Interactive, 2D
- Interactive, 3D

In [ ]:
iss.plot()

In [ ]:
iss.plot(interactive=True)

In [ ]:
iss.plot(use_3d=True, interactive=True)

To create our own `Orbit` object, we can do it using either cartesian or classical elements:

In [ ]:
from poliastro.bodies import Earth
from poliastro.twobody import Orbit

In [ ]:
Orbit.from_vectors(
    Earth,                   # Attractor
    [7000, 0, 0] * u.km,     # Position vector
    [0, 8, 0] * u.km / u.s,  # Velocity vector
)

In [ ]:
from astropy.time import Time

In [ ]:
Orbit.from_classical(
    Earth,
    7000 * u.km,   # Semimajor axis
    0.01 * u.one,  # Eccentricity
    35 * u.deg,    # Inclination
    200 * u.deg,   # Right ascension of the ascending node
    0 * u.deg,     # Argument of periapsis
    90 * u.deg,    # True anomaly,
    Time.now(),    # Epoch (default is J2000)
)

## Let's go to Venus!

![Phosphine in Venus' atmosphere, Nature Astronomy](img/nature-phosphine.png)

Could we go to Venus within two years? To find out, we can use a tool called [Porkchop plots](https://en.wikipedia.org/wiki/Porkchop_plot), available in poliastro.

In [ ]:
from poliastro.plotting.porkchop import PorkchopPlotter
from poliastro.bodies import Earth, Venus
from poliastro.util import time_range

launch_window = time_range("2021-03-21", end="2022-03-21", periods=100)
arrival_window = time_range("2021-11-21", end="2022-06-21", periods=100)

In [ ]:
porkchop_plot = PorkchopPlotter(
    Earth,  # Origin
    Venus,  # Destination
    launch_window,
    arrival_window,
)
(
    dv_dpt,  # Δv departure
    dv_arr,  # Δv arrival
    c3_dpt,   # C3 departure
    c3_arr,   # C3 arrival
    tof      # Time of flight
) = porkchop_plot.porkchop()

The colored contours represent the departure characteristic energy $C_3$ the the red straight lines represent the time of flight (difference between departure and arrival).

Visually, we can see that there are two good launch opportunities around October 2021, the best one with arrival around April 2022. Let us compute the exact dates:

In [ ]:
import numpy as np

def get_minimum(c3, dv_dpt, dv_arr, launch_window, arrival_window):
    """Gets minimum C3 and corresponding launch and arrival dates."""
    index_arrival, index_launch = np.unravel_index(np.nanargmin(c3, axis=None), c3.shape)
    return (
        c3[index_arrival, index_launch],
        dv_dpt[index_arrival, index_launch],
        dv_arr[index_arrival, index_launch],
        launch_window[index_launch],
        arrival_window[index_arrival],
    )

In [ ]:
c3_min, dv_dpt_opt, dv_arr_opt, launch_date, arrival_date = get_minimum(
    c3_dpt, dv_dpt, dv_arr, launch_window, arrival_window
)

In [ ]:
c3_min

In [ ]:
dv_dpt_opt + dv_arr_opt

In [ ]:
launch_date

In [ ]:
arrival_date

### Retrieving the ephemerides of the planets

poliastro has an easy interface to the JPL high precision planetary ephemerides, and can also compute position and velocity of the planets using approximate ones:

In [ ]:
from poliastro.bodies import Sun
from poliastro.ephem import Ephem

In [ ]:
epochs = time_range(launch_date.tdb, end=arrival_date.tdb)

In [ ]:
earth = Ephem.from_body(Earth, epochs)
venus = Ephem.from_body(Venus, epochs)

In [ ]:
earth

### Solving the boundary value problem

Next, let us solve the boundary value problem (Lambert problem). We will neglect the launch and injection and assume that our spacecraft departs with the position and velocity of the Earth at the launch date. Let us extract the initial and final osculating orbits:

In [ ]:
from poliastro.maneuver import Maneuver

In [ ]:
earth_dpt = Orbit.from_ephem(Sun, earth, launch_date.tdb)
earth_dpt

In [ ]:
venus_arr = Orbit.from_ephem(Sun, venus, arrival_date.tdb)
venus_arr

In [ ]:
man = Maneuver.lambert(
    earth_dpt,
    venus_arr,
)
man

Notice that the total cost of the maneuver matches the one we obtained finding the minimum $C_3$ earlier. Now, we can apply this `Maneuver` object to the departure orbit to obtain the transfer arc:

In [ ]:
earth_venus_arc, _ = earth_dpt.apply_maneuver(man, intermediate=True)
earth_venus_arc

### Visualizing everything

Finally, let us visualize the position of the Earth and Venus on departure and arrival, as well as our transfer arc:

In [ ]:
from poliastro.plotting import OrbitPlotter3D
from poliastro.twobody.propagation import propagate

In [ ]:
plotter = OrbitPlotter3D()

plotter.plot_body_orbit(Earth, launch_date.tdb)
plotter.plot_body_orbit(Venus, arrival_date.tdb)
plotter.plot_trajectory(
    propagate(earth_venus_arc, epochs - launch_date),  # Propagate between launch and arrival
    label="Earth-Venus transfer",
    color="black",
)

## Questions

1. If we miss the opportunity window of October 2021, when will be the next one?
2. What is the $\Delta V$ cost of the Lambert transfer for that opportunity?
3. Compute $||\vec{r}||$ of Venus at arrival
4. Using the value above and `Maneuver.hohmann` ([documentation](https://docs.poliastro.space/en/stable/api/safe/maneuver.html#poliastro.maneuver.Maneuver.hohmann)), find out what would be the theoretical minimum cost of a Earth-Venus transfer (this assumes circular and coplanar orbits)